In [ ]:

from numpy import *
set_printoptions(legacy = '1.25')

# activation functions

def relu(z): return 0 if z < 0 else z
def bin(z): return 0 if z < 0 else 1
def sigmoid(z): return 1/(1+exp(-z))
def id(z): return z
# tanh already part of numpy
def one(z): return 1
def zero(z): return 0

# derivative of relu is bin
# derivative of bin is zero
# derivative of s=sigmoid is s*(1-s)
# derivative of id is one
# derivative of tanh is 1-tanh**2

def D_relu(z): return bin(z)
def D_bin(z): return 0
def D_sigmoid(z): return sigmoid(z)*(1-sigmoid(z))
def D_id(z): return 1
def D_tanh(z): return 1 - tanh(z)**2

derivatives = { relu:D_relu, id:D_id, bin:D_bin, sigmoid:D_sigmoid, tanh: D_tanh}


In [ ]:
 
# dtype = 'object' because entries 
# are floats, functions, and strings
	
# network adjacency matrix

w = zeros((8,8), dtype = 'object')	

# indexing in code starts at 0
# nodes are 0, 1, 2, 3, 4, 5, 6, 7
	
w[2][0] = w[2][1] = w[3][0] = w[3][1] = w[4][2] = 1
w[5][2] = w[4][3] = w[5][3] = w[6][4] = w[7][5] = 1
	
w[2][2] = relu
w[3][3] = id
w[4][4] = sigmoid
w[5][5] = tanh

# network weight matrix

w[2][0] = w[2][1] = 0.1
w[3][0] = w[3][1] = -2.0
w[4][2] = w[5][2] = -0.3
w[4][3] = w[5][3] = .22
w[6][4] = w[7][5] = 1


In [ ]:

	
def edges(w): 
	W = full(shape(w), False)
	W[w != 0] = True
	fill_diagonal(W, False)
	return W
	
def outputs(w): 
	return invert(bool(sum(edges(w), axis = 0)))   
	
# these are the non-bias inputs
	
def inputs(w):  
	vector = invert(bool(sum(edges(w), axis = 1)))
	return where(diag(w) == 'b', False, vector) 
	
# bias inputs are here
	
def biases(w):
	vector = full(len(w), False)
	return where(diag(w) == 'b', True, vector) 
	
def neurons(w):	
	vector = full(len(w),True)
	vector = where(diag(w) == 0, False, vector) 
	return where(diag(w) == 'b', False, vector) 


In [ ]:

print(inputs(w))
print(outputs(w))
print(biases(w))
print(neurons(w))


In [ ]:

from numpy.random import default_rng as rng
samples = rng().random
	
# w is a network weight matrix
	
def initial_weights(w, random = 'no'): 
	# set off-diagonal to zero
	W = diag(diag(w))
	if random == 'no': W[edges(w)] = w[edges(w)]
	else: W[edges(w)] = samples(shape(w))[edges(w)]
	return W
	
initial_weights(w, random = 'yes')


In [ ]:

# insert source values at input nodes
# also insert 1 at bias nodes
	
def inject_source(source, w):
	x = full(len(w), None)
	xminus = full(len(w), None)
	x[inputs(w)] = source
	x[biases(w)] = 1
	return xminus, x
	
source = array([1.5,2.5])
xminus, x = inject_source(source, w)
xminus, x


In [ ]:

def incoming(xminus, x, w, i):
	if inputs(w)[i] or biases(w)[i]: return None
	elif xminus[i] != None: return xminus[i]
	else: return sum([ outgoing(xminus, x, w, j) * w[i][j] for j in range(len(w)) if edges(w)[i][j] ])
	
def outgoing(xminus, x, w, i):
	if outputs(w)[i]: return None
	elif x[i] != None: return x[i]
	else: return w[i][i](incoming(xminus, x, w, i))


In [ ]:

def forward_prop(xminus, x, w):
	nodes = arange(len(w))
	# rng.shuffle(nodes)
	for i in nodes: 
		xminus[i] = incoming(xminus, x, w, i)
		x[i]      = outgoing(xminus, x, w, i)
	return xminus, x


In [ ]:
 
xminus, x = inject_source(source, w)	
xminus, x = forward_prop(xminus, x, w)

print(xminus)
print(x)

y = xminus[outputs(w)]
print(y)


In [ ]:

from scipy.special import softmax as sigma

# see terminology remarks at end of section 5.6
from scipy.stats import entropy as I

# both cases

def J(y, target, w, case = 'square'): 
	if case == 'log': 
		# adjust type for case = 'log'
		y = y.astype('float')
		return I(target, sigma(y))
	else: return sum((y - target)**2) / 2
	
target = array([.427,-.288])
y = xminus[outputs(w)]
J(y, target, w)


In [ ]:

def local(xminus, x, w, i): 
	if neurons(w)[i]: return derivatives[w[i][i]](incoming(xminus, x, w, i))
	else: return None


In [ ]:

# returns downstream derivatives of J at output nodes

# both cases

def inject_target(xminus, target, w, case = 'square'):
	delta = full(len(w), None)
	y = xminus[outputs(w)]
	if case == 'log': 
		# adjust type for case = 'log'
		y = y.astype('float')
		q = sigma(y)
	else: q = y
	delta[outputs(w)] = q - target
	return delta


In [ ]:

def downstream(xminus, x, delta, w, j):
	if inputs(w)[j] or biases(w)[j]:  delta[j] = None
	elif delta[j] != None: return delta[j]
	else:
		upstream = sum([ downstream(xminus, x, delta, w, i) * w[i][j] for i in range(len(w)) if edges(w)[i][j] ])
		return upstream * local(xminus,x,w,j)


In [ ]:

def backward_prop(xminus, x, delta, w):
	nodes = arange(len(w))
	for j in nodes: 
		delta[j] = downstream(xminus, x, delta, w, j)
	return delta


In [ ]:
 
delta = inject_target(xminus, target, w)
delta = backward_prop(xminus, x, delta, w)

print(delta)
